In [ ]:
#Importing the required packages
#Ignoring warnings
import warnings
warnings.filterwarnings('ignore') 
import numpy as np
import pandas as pd
from time import time
import operator
import string
import re
import os

import matplotlib.pyplot as plt
from wordcloud import WordCloud,STOPWORDS

import sklearn
from sklearn import utils
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn.metrics import f1_score

import nltk
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords

import tqdm
from tqdm import tqdm, tqdm_notebook
tqdm_notebook().pandas()

import tensorflow as tf
import keras.preprocessing
import keras.layers
import keras.models
from keras import backend as K
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential,Model
from keras.engine.topology import Layer
from keras.layers import Activation,  Wrapper
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Bidirectional, Flatten, SpatialDropout1D, LSTM
from keras.layers import BatchNormalization
from keras.layers import Concatenate
from keras import initializers, regularizers, constraints
from keras.callbacks import (EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard)

In [ ]:
# Retrieving the data from another notebooks
%store -r train_df1
%store -r test_df
%store -r tk
%store -r max_features
%store -r max_len
%store -r word_index
%store -r train_X
%store -r test_X
%store -r train_y
%store -r glove_weights

In [ ]:
# defining the embed size
embed_size = 300

# Path of the best weights model
MODEL_PATH = "weights_best.hdf5"

In [ ]:
# keras attention layer for deep neural networks
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)
        #print("init")

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True
        #print("build")

    def compute_mask(self, input, input_mask=None):
        #print("compute mask")
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        #print("call")
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        #print("compute output shape")
        return input_shape[0],  self.features_dim

In [ ]:
# Keras Wrapper that implements a DropConnect Layer.
class DropConnect(Wrapper):
    def __init__(self, layer, prob=1., **kwargs):
        self.prob = prob
        self.layer = layer
        super(DropConnect, self).__init__(layer, **kwargs)
        if 0. < self.prob < 1.:
            self.uses_learning_phase = True

    def build(self, input_shape):
        if not self.layer.built:
            self.layer.build(input_shape)
            self.layer.built = True
        super(DropConnect, self).build()

    def compute_output_shape(self, input_shape):
        return self.layer.compute_output_shape(input_shape)

    def call(self, x):
        if 0. < self.prob < 1.:
            self.layer.kernel = K.in_train_phase(K.dropout(self.layer.kernel, self.prob), self.layer.kernel)
            self.layer.bias = K.in_train_phase(K.dropout(self.layer.bias, self.prob), self.layer.bias)
        return self.layer.call(x)

In [ ]:
# defining get_model function
def get_model(glove_weights):
    # Input for variable-length sequences of integers
    input_layer = Input(shape=(max_len,))

    # embedding layer
    x = Embedding(max_features, embed_size, weights=[glove_weights], trainable=False)(input_layer)

    # dropout
    x = SpatialDropout1D(rate=0.24)(x)

    # Adding two bidirectional lstm
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    
    # attention layer
    attention = Attention(max_len)(x)
    #attention = BatchNormalization()(attention)
    attention = DropConnect(Dense(64, activation="relu"), prob = 0.05)(attention)
    
    # output layer(sigmoid)
    output_layer = Dense(1, activation="sigmoid")(attention)
    model = Model(inputs=input_layer, outputs=output_layer)

    # compile model
    model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
    return model

In [ ]:
# defining get_callbacks function
def get_callbacks():
    earlystopping = EarlyStopping(monitor='val_loss',
                                  min_delta=0.0001,
                                  patience=2,
                                  verbose=2,
                                  mode='auto')
    checkpoint = ModelCheckpoint(filepath=MODEL_PATH,
                                 monitor='val_loss',
                                 save_best_only=True,
                                 mode='min',
                                 verbose=2)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                  min_lr=0.0001,
                                  factor=0.6,
                                  patience=1,
                                  verbose=2)
    tensorboard = TensorBoard(log_dir='logs1/{}'.format(time()))
    
    return [earlystopping, checkpoint, reduce_lr, tensorboard]

In [ ]:
best_thres = []
y_test = np.zeros((test_X.shape[0], ))        

# splitting the train data into train and validation data using stratified sampling
X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, stratify=train_y, test_size=0.2, random_state=38) 

# calling the get_model function
model = get_model(glove_weights)
print(model.summary())


# get class weight
weights = None
weights = utils.class_weight.compute_class_weight('balanced', np.unique(y_train), y_train) 


# training the model
history = model.fit(X_train, y_train, batch_size=512, epochs=5, 
          validation_data=(X_val, y_val), 
          verbose=2, class_weight=weights, callbacks=get_callbacks())



In [ ]:
# reload best model
model.load_weights(MODEL_PATH)

# predicting the target class values for validation data using the trained model
pred_y_val = model.predict([X_val], batch_size=1024, verbose=2)

# calculating and displaying the f1 score at respective thresholds
thresholds = []
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    res = metrics.f1_score(np.squeeze(y_val), (np.squeeze(pred_y_val)>thresh).astype(int))
    thresholds.append([thresh, res])
    print("F1 score at threshold {0} is {1}".format(thresh, res))

# displaying the best threshold    
thresholds.sort(key=lambda x: x[1], reverse=True)
best_thresh = thresholds[0][0]
print("Best threshold: ", best_thresh)

# predicting the target class values for test data using the trained model
y_test = y_test + np.squeeze(model.predict([test_X], batch_size=1024, verbose=2))
    

In [ ]:
# Storing the predicted target values for test dataset into dataframe
y_test = y_test.reshape((-1, 1))
test_df['prediction'] = (y_test>np.mean(best_thresh)).astype(int)

In [ ]:
# Confusion matrix for validation data
from sklearn.metrics import confusion_matrix
confusion_matrix(y_val, (pred_y_val>np.mean(best_thresh)).astype(int))

In [ ]:
# Confusion matrix for test data
confusion_matrix(test_y, (y_test>np.mean(best_thresh)).astype(int))

In [ ]:
# plot training history of Train and Validation Loss During model Training
from matplotlib import pyplot
pyplot.figure(figsize=(9,7))
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='validation')
pyplot.title('Line Plot of Train and Validation Loss During Training With Patient Early Stopping')
pyplot.legend()
pyplot.xlabel('epochs')
pyplot.ylabel('loss')
pyplot.show()

In [ ]:
# plot training history of Train and Validation accuracy During model Training
pyplot.figure(figsize=(9,7))
pyplot.plot(history.history['acc'], label='train')
pyplot.plot(history.history['val_acc'], label='validation')
pyplot.title('Line Plot of Train and Validation accuracy During Training With Patient Early Stopping')
pyplot.legend()
pyplot.xlabel('epochs')
pyplot.ylabel('accuracy')
pyplot.show()

In [ ]:
#Records with both target and predicted value = '1'
test_df[(test_df.target==1)&(test_df.prediction==1)]

In [ ]:
#Displaying the classification report 
from sklearn.metrics import classification_report
target_names = ['Sincere', 'Insincere']
print(classification_report(test_y, (y_test>np.mean(best_thresh)).astype(int), target_names=target_names))